# 🦖 DaZZLeD: Recursive Hasher Training Notebook

**Goal:** Train a Tiny Recursive Model (TRM) using DINOv3 distillation for adversarially-robust perceptual hashing.

## How to Use This Notebook

1. **Run Cell 1** - Mount Google Drive
2. **Run Cell 2** - Download training datasets (one-time, ~30 min)
3. **Run Cell 3** - Clone repo & install dependencies
4. **Run Cell 4** - Build manifest & train the model
5. **Run remaining cells** - Test and export the trained model

⚠️ **Before training:** Runtime → Change runtime type → **GPU (T4 free or A100)**

## 0. Mount Google Drive

In [ ]:
# Mount Google Drive (required for data storage)
from google.colab import drive
drive.mount('/content/drive')

# Create project directories
from pathlib import Path

DRIVE_ROOT = Path("/content/drive/MyDrive/dazzled")
DATA_ROOT = DRIVE_ROOT / "data"
OUTPUT_ROOT = DRIVE_ROOT / "outputs"

# Create all needed directories
for d in [DATA_ROOT / "ffhq", DATA_ROOT / "openimages", DATA_ROOT / "text",
          OUTPUT_ROOT / "checkpoints", OUTPUT_ROOT / "models",
          DRIVE_ROOT / "manifests"]:
    d.mkdir(parents=True, exist_ok=True)

print(f"✓ Project root: {DRIVE_ROOT}")
print(f"✓ Data root: {DATA_ROOT}")
print(f"✓ Output root: {OUTPUT_ROOT}")

## 0.1 Download Training Datasets (Direct to Drive)

Run this cell **once** to download datasets directly to Google Drive. Takes ~30 minutes.

**Datasets used (non-sensitive proxies):**
- **FFHQ Thumbnails** (10k faces) - Human-centric feature extraction
- **OpenImages Subset** (30k images) - Diverse real-world images  
- **COCO Subset** (5k images) - Additional diversity

⚠️ Skip this cell if you've already downloaded the data.

In [ ]:
# =============================================================================
# DOWNLOAD DATASETS DIRECTLY TO GOOGLE DRIVE
# Run this cell ONCE - it will download ~5GB of training data
# =============================================================================

import os
import zipfile
import tarfile
from pathlib import Path
import urllib.request
import shutil

DATA_ROOT = Path("/content/drive/MyDrive/dazzled/data")

def download_with_progress(url, dest):
    """Download file with progress indicator."""
    def reporthook(count, block_size, total_size):
        percent = min(100, int(count * block_size * 100 / total_size))
        print(f"\r  Downloading: {percent}%", end="", flush=True)
    
    urllib.request.urlretrieve(url, dest, reporthook)
    print(" ✓")

# -----------------------------------------------------------------------------
# 1. FFHQ Thumbnails (128x128, 10k images) - ~95MB
# -----------------------------------------------------------------------------
ffhq_dir = DATA_ROOT / "ffhq"
if len(list(ffhq_dir.glob("*.png"))) < 1000:
    print("📥 Downloading FFHQ thumbnails...")
    
    # FFHQ thumbnails from official Google Drive
    # Using a smaller 10k subset for training
    !pip install -q gdown
    import gdown
    
    # FFHQ thumbnails 128x128 (official)
    ffhq_url = "https://drive.google.com/uc?id=1SbZKLwDxBV2k4aFa5LS9lEE3cNMV5IEU"
    ffhq_zip = "/content/ffhq_thumbnails.zip"
    
    gdown.download(ffhq_url, ffhq_zip, quiet=False)
    
    print("  Extracting FFHQ...")
    with zipfile.ZipFile(ffhq_zip, 'r') as z:
        z.extractall("/content/ffhq_temp")
    
    # Move images to Drive and resize to 224x224
    from PIL import Image
    from torchvision import transforms
    
    resize_transform = transforms.Resize((224, 224))
    
    temp_images = list(Path("/content/ffhq_temp").rglob("*.png"))[:10000]
    for i, img_path in enumerate(temp_images):
        if i % 1000 == 0:
            print(f"  Processing FFHQ: {i}/{len(temp_images)}")
        try:
            img = Image.open(img_path).convert("RGB")
            img = resize_transform(img)
            img.save(ffhq_dir / f"ffhq_{i:05d}.jpg", quality=95)
        except:
            pass
    
    # Cleanup
    os.remove(ffhq_zip)
    shutil.rmtree("/content/ffhq_temp", ignore_errors=True)
    print(f"✓ FFHQ: {len(list(ffhq_dir.glob('*.jpg')))} images")
else:
    print(f"✓ FFHQ already exists: {len(list(ffhq_dir.glob('*')))} images")

# -----------------------------------------------------------------------------
# 2. COCO 2017 Validation Set (5k images) - ~1GB
# -----------------------------------------------------------------------------
coco_dir = DATA_ROOT / "openimages"  # Using for general images
if len(list(coco_dir.glob("*.jpg"))) < 1000:
    print("\n📥 Downloading COCO 2017 validation set...")
    
    coco_url = "http://images.cocodataset.org/zips/val2017.zip"
    coco_zip = "/content/coco_val2017.zip"
    
    download_with_progress(coco_url, coco_zip)
    
    print("  Extracting COCO...")
    with zipfile.ZipFile(coco_zip, 'r') as z:
        z.extractall("/content/coco_temp")
    
    # Resize and move to Drive
    from PIL import Image
    from torchvision import transforms
    
    resize_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
    ])
    
    temp_images = list(Path("/content/coco_temp/val2017").glob("*.jpg"))
    for i, img_path in enumerate(temp_images):
        if i % 1000 == 0:
            print(f"  Processing COCO: {i}/{len(temp_images)}")
        try:
            img = Image.open(img_path).convert("RGB")
            img = resize_transform(img)
            img.save(coco_dir / img_path.name, quality=95)
        except:
            pass
    
    # Cleanup
    os.remove(coco_zip)
    shutil.rmtree("/content/coco_temp", ignore_errors=True)
    print(f"✓ COCO: {len(list(coco_dir.glob('*.jpg')))} images")
else:
    print(f"✓ COCO already exists: {len(list(coco_dir.glob('*')))} images")

# -----------------------------------------------------------------------------
# 3. Text/Document Images (synthetic screenshots)
# -----------------------------------------------------------------------------
text_dir = DATA_ROOT / "text"
if len(list(text_dir.glob("*.jpg"))) < 100:
    print("\n📥 Generating synthetic text images...")
    
    from PIL import Image, ImageDraw, ImageFont
    import random
    import string
    
    # Generate simple text images as edge cases
    for i in range(500):
        img = Image.new('RGB', (224, 224), color=(255, 255, 255))
        draw = ImageDraw.Draw(img)
        
        # Random text
        text = ''.join(random.choices(string.ascii_letters + string.digits + ' ', k=random.randint(20, 100)))
        
        # Wrap text
        lines = [text[j:j+20] for j in range(0, len(text), 20)]
        y = 20
        for line in lines[:8]:
            draw.text((10, y), line, fill=(0, 0, 0))
            y += 25
        
        img.save(text_dir / f"text_{i:04d}.jpg", quality=95)
    
    print(f"✓ Text images: {len(list(text_dir.glob('*.jpg')))} images")
else:
    print(f"✓ Text images already exist: {len(list(text_dir.glob('*')))} images")

# -----------------------------------------------------------------------------
# Summary
# -----------------------------------------------------------------------------
print("\n" + "="*50)
print("📊 DATASET SUMMARY")
print("="*50)
total = 0
for name, path in [("FFHQ (faces)", DATA_ROOT / "ffhq"), 
                   ("OpenImages/COCO", DATA_ROOT / "openimages"),
                   ("Text/Docs", DATA_ROOT / "text")]:
    count = len(list(path.glob("*")))
    total += count
    print(f"  {name}: {count:,} images")
print(f"  {'─'*30}")
print(f"  TOTAL: {total:,} images")
print("="*50)

## 1. Setup & Installation

In [ ]:
# Clone the repo (only needed in Colab)
import os
if not os.path.exists('DaZZLeD'):
    !git clone https://github.com/D13ya/DaZZLeD.git
    %cd DaZZLeD/ml-core
else:
    %cd DaZZLeD/ml-core

!pip install -q -r requirements.txt

In [ ]:
import sys
import torch
import torch.nn.functional as F
from pathlib import Path

# Add project root to path
sys.path.insert(0, str(Path.cwd()))

from models.recursive_student import RecursiveHasher

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 1.1 Build Manifest & Train Model

⚠️ **Switch to GPU first:** Runtime → Change runtime type → GPU (T4 or A100)

This cell:
1. Builds a manifest of all training images
2. Trains the RecursiveHasher using DINOv3 distillation
3. Saves checkpoints to Google Drive

In [ ]:
# =============================================================================
# BUILD MANIFEST FROM DOWNLOADED DATA
# =============================================================================

from pathlib import Path

DRIVE_ROOT = Path("/content/drive/MyDrive/dazzled")
DATA_ROOT = DRIVE_ROOT / "data"

# Find all training images
exts = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}
paths = []

for data_dir in [DATA_ROOT / "ffhq", DATA_ROOT / "openimages", DATA_ROOT / "text"]:
    if data_dir.exists():
        paths.extend([str(p) for p in data_dir.rglob("*") if p.suffix.lower() in exts])

print(f"Found {len(paths):,} training images")

if len(paths) < 100:
    print("⚠️  Not enough images! Run the download cell first.")
else:
    # Write manifest
    manifest = DRIVE_ROOT / "manifests/train.txt"
    manifest.parent.mkdir(parents=True, exist_ok=True)
    manifest.write_text("\n".join(paths))
    print(f"✓ Manifest written: {manifest}")

In [ ]:
# =============================================================================
# TRAIN THE MODEL (Choose one option)
# =============================================================================

# 🔧 TRAINING OPTIONS - Uncomment ONE of the following:

# -----------------------------------------------------------------------------
# OPTION A: Quick Test (5-10 min on T4) - Just to verify everything works
# -----------------------------------------------------------------------------
!python training/train.py \
    --data-list /content/drive/MyDrive/dazzled/manifests/train.txt \
    --teacher facebook/dinov3-vitl16-pretrain-lvd1689m \
    --epochs 1 \
    --batch-size 16 \
    --recursion-steps 8 \
    --max-steps 100 \
    --amp \
    --log-interval 10 \
    --checkpoint-dir /content/drive/MyDrive/dazzled/outputs/checkpoints

# -----------------------------------------------------------------------------
# OPTION B: Full Training (2-4 hours on A100) - Production quality
# -----------------------------------------------------------------------------
# !python training/train.py \
#     --data-list /content/drive/MyDrive/dazzled/manifests/train.txt \
#     --teacher facebook/dinov3-vitl16-pretrain-lvd1689m \
#     --epochs 5 \
#     --batch-size 64 \
#     --recursion-steps 16 \
#     --grad-accum 2 \
#     --lr 1e-4 \
#     --amp \
#     --allow-tf32 \
#     --channels-last \
#     --cudnn-benchmark \
#     --workers 4 \
#     --log-interval 50 \
#     --checkpoint-dir /content/drive/MyDrive/dazzled/outputs/checkpoints \
#     --checkpoint-every 500

# -----------------------------------------------------------------------------
# OPTION C: Resume from Checkpoint (if session disconnected)
# -----------------------------------------------------------------------------
# !python training/train.py \
#     --data-list /content/drive/MyDrive/dazzled/manifests/train.txt \
#     --teacher facebook/dinov3-vitl16-pretrain-lvd1689m \
#     --resume /content/drive/MyDrive/dazzled/outputs/checkpoints/student_epoch_3.safetensors \
#     --epochs 5 \
#     --batch-size 64 \
#     --recursion-steps 16 \
#     --amp \
#     --checkpoint-dir /content/drive/MyDrive/dazzled/outputs/checkpoints

In [ ]:
# =============================================================================
# LIST CHECKPOINTS & LOAD TRAINED WEIGHTS
# =============================================================================

from pathlib import Path
import safetensors.torch

CKPT_DIR = Path("/content/drive/MyDrive/dazzled/outputs/checkpoints")

# List available checkpoints
checkpoints = sorted(CKPT_DIR.glob("*.safetensors"))
print(f"Found {len(checkpoints)} checkpoints:")
for ckpt in checkpoints:
    size_mb = ckpt.stat().st_size / (1024 * 1024)
    print(f"  {ckpt.name} ({size_mb:.2f} MB)")

# Load the latest checkpoint into model
if checkpoints:
    latest_ckpt = checkpoints[-1]
    print(f"\n📥 Loading: {latest_ckpt.name}")
    
    # Model should already be defined from earlier cell
    # If not, uncomment:
    # from models.recursive_student import RecursiveHasher
    # model = RecursiveHasher(state_dim=128, hash_dim=96)
    
    safetensors.torch.load_model(model, str(latest_ckpt))
    model.eval()
    print("✓ Trained weights loaded!")
else:
    print("⚠️  No checkpoints found. Run training first.")

## 2. Model Architecture Test

In [ ]:
# Initialize model with default params
STATE_DIM = 128
HASH_DIM = 96
RECURSION_STEPS = 16

model = RecursiveHasher(state_dim=STATE_DIM, hash_dim=HASH_DIM)
model.eval()

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Model size: ~{total_params * 4 / 1024 / 1024:.2f} MB (float32)")

In [ ]:
# Test forward pass with dummy input
batch_size = 4
image_size = 224

dummy_img = torch.randn(batch_size, 3, image_size, image_size)
dummy_state = torch.zeros(batch_size, STATE_DIM)

with torch.no_grad():
    next_state, hash_out = model(dummy_img, dummy_state)

print(f"Input image shape: {dummy_img.shape}")
print(f"Input state shape: {dummy_state.shape}")
print(f"Output state shape: {next_state.shape}")
print(f"Output hash shape: {hash_out.shape}")
print(f"Hash L2 norm (should be ~1.0): {torch.norm(hash_out, dim=1).mean():.4f}")

## 3. Recursive Inference Test

The key innovation is running the model recursively 16 times, refining the hash at each step.

In [ ]:
def recursive_inference(model, image, steps=16):
    """Run recursive inference for the specified number of steps."""
    batch_size = image.size(0)
    state = torch.zeros(batch_size, STATE_DIM, device=image.device)
    
    hashes = []
    with torch.no_grad():
        for step in range(steps):
            state, hash_out = model(image, state)
            hashes.append(hash_out.clone())
    
    return hashes

# Run recursive inference
hashes = recursive_inference(model, dummy_img, steps=RECURSION_STEPS)

print(f"Generated {len(hashes)} hash vectors")
print(f"Final hash shape: {hashes[-1].shape}")

In [ ]:
# Analyze hash stability across recursive steps
import matplotlib.pyplot as plt

# Compute cosine similarity between consecutive steps
similarities = []
for i in range(1, len(hashes)):
    sim = F.cosine_similarity(hashes[i], hashes[i-1], dim=1).mean().item()
    similarities.append(sim)

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(hashes)), similarities, 'b-o')
plt.xlabel('Recursion Step')
plt.ylabel('Cosine Similarity with Previous')
plt.title('Hash Convergence Over Recursion')
plt.ylim(0, 1.1)
plt.grid(True)

# Compute similarity to final hash
final_hash = hashes[-1]
similarities_to_final = []
for h in hashes:
    sim = F.cosine_similarity(h, final_hash, dim=1).mean().item()
    similarities_to_final.append(sim)

plt.subplot(1, 2, 2)
plt.plot(range(len(hashes)), similarities_to_final, 'r-o')
plt.xlabel('Recursion Step')
plt.ylabel('Cosine Similarity to Final Hash')
plt.title('Convergence to Final Hash')
plt.ylim(0, 1.1)
plt.grid(True)

plt.tight_layout()
plt.show()

## 4. Adversarial Robustness Test

Test if small perturbations to input cause large changes in hash (they shouldn't after recursive refinement).

In [ ]:
def test_perturbation_robustness(model, image, epsilon_range, steps=16):
    """Test hash stability under input perturbations."""
    results = []
    
    # Get baseline hash
    baseline_hashes = recursive_inference(model, image, steps)
    baseline_final = baseline_hashes[-1]
    
    for epsilon in epsilon_range:
        # Add random noise
        noise = torch.randn_like(image) * epsilon
        perturbed = image + noise
        
        # Get perturbed hash
        perturbed_hashes = recursive_inference(model, perturbed, steps)
        perturbed_final = perturbed_hashes[-1]
        
        # Compute similarity
        sim = F.cosine_similarity(baseline_final, perturbed_final, dim=1).mean().item()
        results.append((epsilon, sim))
        print(f"Epsilon={epsilon:.4f}: Cosine Similarity={sim:.4f}")
    
    return results

# Test with various noise levels
epsilons = [0.001, 0.01, 0.05, 0.1, 0.2, 0.5]
test_img = torch.randn(1, 3, 224, 224)
results = test_perturbation_robustness(model, test_img, epsilons)

In [ ]:
# Plot robustness results
epsilons, sims = zip(*results)

plt.figure(figsize=(8, 5))
plt.plot(epsilons, sims, 'g-o', linewidth=2, markersize=8)
plt.axhline(y=0.9, color='r', linestyle='--', label='0.9 threshold')
plt.xlabel('Noise Level (epsilon)', fontsize=12)
plt.ylabel('Cosine Similarity to Original', fontsize=12)
plt.title('Hash Robustness to Input Perturbations', fontsize=14)
plt.xscale('log')
plt.ylim(0, 1.1)
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

## 5. ONNX Export Test

In [ ]:
import onnx
import onnxruntime as ort
import numpy as np

# Export to ONNX
onnx_path = "test_model.onnx"

model.eval()
dummy_img = torch.randn(1, 3, 224, 224)
dummy_state = torch.zeros(1, STATE_DIM)

torch.onnx.export(
    model,
    (dummy_img, dummy_state),
    onnx_path,
    input_names=["image", "prev_state"],
    output_names=["next_state", "hash"],
    opset_version=14,
    dynamic_axes={
        "image": {0: "batch"},
        "prev_state": {0: "batch"},
        "next_state": {0: "batch"},
        "hash": {0: "batch"},
    },
)

print(f"Exported ONNX model to {onnx_path}")
print(f"File size: {os.path.getsize(onnx_path) / 1024:.2f} KB")

In [ ]:
# Validate ONNX model
onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)
print("ONNX model validation passed!")

# Test ONNX runtime inference
session = ort.InferenceSession(onnx_path)

# Run inference
test_img = np.random.randn(1, 3, 224, 224).astype(np.float32)
test_state = np.zeros((1, STATE_DIM), dtype=np.float32)

outputs = session.run(None, {"image": test_img, "prev_state": test_state})
next_state_onnx, hash_onnx = outputs

print(f"ONNX output state shape: {next_state_onnx.shape}")
print(f"ONNX output hash shape: {hash_onnx.shape}")
print(f"ONNX hash L2 norm: {np.linalg.norm(hash_onnx, axis=1).mean():.4f}")

In [ ]:
# Compare PyTorch vs ONNX outputs
with torch.no_grad():
    pt_state, pt_hash = model(torch.from_numpy(test_img), torch.from_numpy(test_state))

pt_hash_np = pt_hash.numpy()
pt_state_np = pt_state.numpy()

hash_diff = np.abs(pt_hash_np - hash_onnx).max()
state_diff = np.abs(pt_state_np - next_state_onnx).max()

print(f"Max hash difference (PyTorch vs ONNX): {hash_diff:.8f}")
print(f"Max state difference (PyTorch vs ONNX): {state_diff:.8f}")

if hash_diff < 1e-5 and state_diff < 1e-5:
    print("✅ ONNX export matches PyTorch output!")
else:
    print("⚠️ Warning: Numerical differences detected")

## 6. Latency Benchmark

In [ ]:
import time

def benchmark_inference(model, device, num_runs=100, warmup=10):
    """Benchmark recursive inference latency."""
    model = model.to(device)
    model.eval()
    
    test_img = torch.randn(1, 3, 224, 224).to(device)
    
    # Warmup
    for _ in range(warmup):
        _ = recursive_inference(model, test_img, steps=16)
    
    if device.type == 'cuda':
        torch.cuda.synchronize()
    
    # Benchmark
    times = []
    for _ in range(num_runs):
        if device.type == 'cuda':
            torch.cuda.synchronize()
        start = time.perf_counter()
        
        _ = recursive_inference(model, test_img, steps=16)
        
        if device.type == 'cuda':
            torch.cuda.synchronize()
        times.append(time.perf_counter() - start)
    
    return times

# Benchmark on CPU
cpu_times = benchmark_inference(model, torch.device('cpu'), num_runs=50)
print(f"CPU Latency: {np.mean(cpu_times)*1000:.2f} ± {np.std(cpu_times)*1000:.2f} ms")

# Benchmark on GPU if available
if torch.cuda.is_available():
    gpu_times = benchmark_inference(model, torch.device('cuda'), num_runs=100)
    print(f"GPU Latency: {np.mean(gpu_times)*1000:.2f} ± {np.std(gpu_times)*1000:.2f} ms")

## 8. Summary

### ✅ This notebook provides a complete workflow:

| Step | Cell | Description |
|------|------|-------------|
| 0 | Mount Drive | Connect Google Drive for data/checkpoints |
| 0.1 | Download Data | Download FFHQ, COCO, text images directly to Drive |
| 1 | Setup | Clone repo & install dependencies |
| 1.1 | Train | Build manifest & train with DINOv3 distillation |
| 2-3 | Architecture | Verify model structure & forward pass |
| 4 | Recursion | Test recursive inference convergence |
| 5 | Robustness | Perturbation stability testing |
| 6 | ONNX | Export & validate ONNX model |
| 7 | Save | Copy ONNX to Drive for Go runtime |

### 📦 Output artifacts (on Google Drive):

```
/content/drive/MyDrive/dazzled/outputs/
├── checkpoints/
│   └── student_epoch_5.safetensors   # Trained weights
└── models/
    └── recursive_hasher.onnx          # ONNX for Go runtime
```

### 🔗 References:
- **DINOv3:** [arXiv:2508.10104](https://arxiv.org/abs/2508.10104)
- **TRM:** [arXiv:2510.04871](https://arxiv.org/abs/2510.04871)
- **Split Accumulation:** [ePrint 2020/1618](https://eprint.iacr.org/2020/1618)

## 7. Export ONNX to Google Drive

Save the trained model to Drive for use in the Go application.

In [ ]:
# =============================================================================
# EXPORT TRAINED ONNX MODEL TO GOOGLE DRIVE
# =============================================================================

import shutil
from pathlib import Path

# Source: the ONNX file exported earlier in this notebook
source_onnx = Path("test_model.onnx")

# Destination on Drive
dest_dir = Path("/content/drive/MyDrive/dazzled/outputs/models")
dest_dir.mkdir(parents=True, exist_ok=True)
dest_onnx = dest_dir / "recursive_hasher.onnx"

if source_onnx.exists():
    shutil.copy(source_onnx, dest_onnx)
    size_mb = dest_onnx.stat().st_size / (1024 * 1024)
    print(f"✓ ONNX model saved to Drive: {dest_onnx}")
    print(f"  Size: {size_mb:.2f} MB")
    
    # Also save the latest checkpoint
    ckpt_dir = Path("/content/drive/MyDrive/dazzled/outputs/checkpoints")
    checkpoints = sorted(ckpt_dir.glob("*.safetensors"))
    if checkpoints:
        print(f"\n📦 Available artifacts on Drive:")
        print(f"  Model: {dest_onnx}")
        print(f"  Checkpoint: {checkpoints[-1]}")
else:
    print("⚠️  ONNX file not found. Run the ONNX export cell first (Cell 23-25).")